In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-monthly.csv || wget  https://github.com/heiseacademy/ml-python/raw/main/02-ml-intro/heise-monthly.csv")

In [ ]:
import pandas as pd
heise_monthly = pd.read_csv("heise-monthly.csv", parse_dates=["month"], index_col="month")

In [ ]:
heise_monthly["count"].plot()

In [ ]:
from scipy.stats import linregress
lrc = linregress(range(len(heise_monthly)), heise_monthly["count"].values)
lrc

In [ ]:
heise_monthly["predict_count"] = [i*lrc.slope+lrc.intercept for i in range(len(heise_monthly))]

In [ ]:
heise_monthly[["count", "predict_count"]].plot()

In [ ]:
lrt = linregress(range(len(heise_monthly)), heise_monthly["tokens"].values)
lrt

In [ ]:
heise_monthly["predict_tokens"] = [i*lrt.slope+lrt.intercept for i in range(len(heise_monthly))]

In [ ]:
heise_monthly[["tokens", "predict_tokens"]].plot()

In [ ]:
from sklearn import linear_model
slrt = linear_model.LinearRegression()
X = [[i] for i in range(len(heise_monthly))]
Y = heise_monthly["tokens"].values
slrt.fit(X, Y)

In [ ]:
heise_monthly["predict_tokens_sklearn_linear"] = slrt.predict(X)

In [ ]:
heise_monthly[["tokens", "predict_tokens_sklearn_linear"]].plot()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
def print_scores(ground_truth, predict):
    print('Mittlerer quadratischer Fehler', mean_squared_error(ground_truth, predict))
    # 1 entspricht einer 100% Vorhersage
    print('Coefficient of determination: %.2f' % r2_score(ground_truth, predict))
    
print_scores(Y, heise_monthly["predict_tokens_sklearn_linear"])

In [ ]:
(X_train, X_test) = (X[:-50], X[-50:])
(Y_train, Y_test) = (Y[:-50], Y[-50:])
slrt.fit(X_train, Y_train)

In [ ]:
print_scores(Y_test, slrt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_linear"] = slrt.predict(X)

In [ ]:
heise_monthly["predict_tokens_sklearn_linear_train"] = list(slrt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_linear_test"] = [None]*len(X_train) + list(slrt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_linear_train", "predict_tokens_sklearn_linear_test"]].plot()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtt = DecisionTreeRegressor(max_depth=4)

dtt.fit(X_train, Y_train)
print_scores(Y_test, dtt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_dt_train"] = list(dtt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_dt_test"] = [None]*len(X_train) + list(dtt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_dt_train", "predict_tokens_sklearn_dt_test"]].plot()

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

abt = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=42)

abt.fit(X_train, Y_train)
print_scores(Y_test, abt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_ab_train"] = list(abt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_ab_test"] = [None]*len(X_train) + list(abt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_ab_train", "predict_tokens_sklearn_ab_test"]].plot()

In [ ]:
from sklearn import ensemble
gbt = ensemble.GradientBoostingRegressor()
gbt.fit(X_train, Y_train)
print_scores(Y_test, gbt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_gb_train"] = list(gbt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_gb_test"] = [None]*len(X_train) + list(gbt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_gb_train", "predict_tokens_sklearn_gb_test"]].plot()

In [ ]:
%pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
pa = pd.DataFrame()
pa["ds"] = heise_monthly.index.values
pa["y"] = heise_monthly["tokens"].values
pa

In [ ]:
m = Prophet()
m.fit(pa)

In [ ]:
future = m.make_future_dataframe(periods=20, freq='M')

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)